# Part I: CNN

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D  
from keras.layers import MaxPooling2D   
from keras.layers import Flatten        
from keras.layers import Dense          
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import warnings 
warnings.filterwarnings("ignore")

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

training_set = train_datagen.flow_from_directory(
    r"C:\Users\visha\Downloads\Shoe Classification\Shoe Classification\train", 
    target_size=(136, 102),
    batch_size=32,
    class_mode='categorical',  
    subset='training'
)

validation_set = train_datagen.flow_from_directory(
    r"C:\Users\visha\Downloads\Shoe Classification\Shoe Classification\train",  
    target_size=(136, 102),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 623 images belonging to 3 classes.
Found 155 images belonging to 3 classes.


In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), input_shape=(136, 102, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=len(training_set.class_indices), activation='softmax')  

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
model.fit(
    training_set,
    steps_per_epoch=training_set.samples // training_set.batch_size,
    epochs=10,
    validation_data=validation_set,
    validation_steps=validation_set.samples // validation_set.batch_size
)

Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 358ms/step - accuracy: 0.5489 - loss: 1.1973 - val_accuracy: 0.7812 - val_loss: 0.4680
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7812 - loss: 0.3674 - val_accuracy: 0.5926 - val_loss: 0.8179
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 208ms/step - accuracy: 0.8234 - loss: 0.3961 - val_accuracy: 0.9062 - val_loss: 0.3262
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9688 - loss: 0.1972 - val_accuracy: 1.0000 - val_loss: 0.1816
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 226ms/step - accuracy: 0.9114 - loss: 0.2359 - val_accuracy: 0.8984 - val_loss: 0.3036
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9062 - loss: 0.1739 - val_accuracy: 0.9630 - val_loss: 0.2483
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 213ms/step - accuracy: 0.9350 - loss: 0.1725 - val_accuracy: 0.9141 - val_loss: 0.2706
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9688 - loss: 0.1246 - val_accuracy: 0.9259 -

In [6]:
test_folder =r"C:\Users\visha\Downloads\Shoe Classification\Shoe Classification\test"
test_images = []
test_filenames = []

for filename in os.listdir(test_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):  
        img_path = os.path.join(test_folder, filename)
        img = image.load_img(img_path, target_size=(136, 102))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0  # Rescale
        test_images.append(img_array)
        test_filenames.append(filename)

In [7]:
test_images = np.vstack(test_images)

predictions = model.predict(test_images)

class_labels = {v: k for k, v in training_set.class_indices.items()}
predicted_classes = [class_labels[np.argmax(prediction)] for prediction in predictions]

for i, predicted_class in enumerate(predicted_classes):
    print(f"Image: {test_filenames[i]}, Predicted Class: {predicted_class}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Image: image (1).jpg, Predicted Class: Sandals
Image: image (10).jpg, Predicted Class: Boots
Image: image (11).jpg, Predicted Class: Boots
Image: image (12).jpg, Predicted Class: Slippers
Image: image (13).jpg, Predicted Class: Boots
Image: image (14).jpg, Predicted Class: Sandals
Image: image (15).jpg, Predicted Class: Boots
Image: image (16).jpg, Predicted Class: Sandals
Image: image (17).jpg, Predicted Class: Sandals
Image: image (18).jpg, Predicted Class: Boots
Image: image (19).jpg, Predicted Class: Boots
Image: image (2).jpg, Predicted Class: Boots
Image: image (20).jpg, Predicted Class: Boots
Image: image (21).jpg, Predicted Class: Boots
Image: image (22).jpg, Predicted Class: Boots
Image: image (23).jpg, Predicted Class: Boots
Image: image (24).jpg, Predicted Class: Sandals
Image: image (25).jpg, Predicted Class: Sandals
Image: image (26).jpg, Predicted Class: Sandals
Image: image (27).jpg, Predicted Class: Sandals
Image: image (28).jpg, Pr

# Part II: RNN

In [8]:
from numpy import array
import pandas as pd

In [9]:
data = pd.read_csv(r"C:\Users\visha\Downloads\Tweets.csv")
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [10]:
data = data[['text', 'airline_sentiment']]
data.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [11]:
data.text= data.text.str.lower()           
data.head()

,text,airline_sentiment
0,@virginamerica what @dhepburn said.,neutral
1,@virginamerica plus you've added commercials t...,positive
2,@virginamerica i didn't today... must mean i n...,neutral
3,@virginamerica it's really aggressive to blast...,negative
4,@virginamerica and it's a really big bad thing...,negative


In [12]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
def remove_punctuation(text):
    text_nopunt="".join([c 
                         for c in text
                         if c not in string.punctuation])
    return text_nopunt

In [14]:
data.iloc[15,0]

'@virginamerica sfo-pdx schedule is still mia.'

In [16]:
data.text = data.text.apply(lambda x : remove_punctuation(x))
data.iloc[15,0]

'virginamerica sfopdx schedule is still mia'

In [17]:
data.iloc[26,0]
data['text'] = data['text'].replace(r'\d+', '', regex=True)
data.iloc[26,0]

'virginamerica what happened  ur vegan food options at least say on ur site so i know i wont be able  eat anything for next  hrs fail'

In [19]:
import emoji

In [20]:
data.iloc[24,0]
def convert_emojis_to_text(text):
    return emoji.demojize(text, language='en')

data['text'] = data['text'].apply(convert_emojis_to_text)
data.iloc[24,0]

'virginamerica you guys messed up my seating i reserved seating with my friends and you guys gave my seat away  :enraged_face: i want free internet'

In [21]:
data.text = data.text.apply(lambda x : remove_punctuation(x))
data.iloc[24,0]

'virginamerica you guys messed up my seating i reserved seating with my friends and you guys gave my seat away  enragedface i want free internet'

In [22]:
data.shape
data =  data.drop_duplicates()
data.shape

(14392, 2)

In [23]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

data['text'] = data['text'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\visha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
sentiment_mapping = {'positive': 0, 'neutral': 1, 'negative': 2}
data['airline_sentiment'] = data['airline_sentiment'].map(sentiment_mapping)

In [25]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['text'])
vocab_size = len(tokenizer.word_index) + 1

In [26]:
sequences = tokenizer.texts_to_sequences(data['text'])

In [27]:
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

In [28]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, data['airline_sentiment'], test_size=0.2, random_state=42)

In [29]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 100),  
    tf.keras.layers.SimpleRNN(64),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [30]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
144/144 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.6552 - loss: 0.7977 - val_accuracy: 0.7121 - val_loss: 0.6718
Epoch 2/5
144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.8487 - loss: 0.4051 - val_accuracy: 0.7290 - val_loss: 0.6782
Epoch 3/5
144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9382 - loss: 0.1881 - val_accuracy: 0.7238 - val_loss: 0.8098
Epoch 4/5
144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9794 - loss: 0.0727 - val_accuracy: 0.7173 - val_loss: 0.9653
Epoch 5/5
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.9857 - loss: 0.0529 - val_accuracy: 0.7039 - val_loss: 1.0111


In [32]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7091 - loss: 0.9810
Test Accuracy: 0.7103160619735718


In [33]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report


y_pred = np.argmax(model.predict(X_test), axis=-1)
conf_matrix = confusion_matrix(y_test, y_pred)


print("Confusion Matrix:")
print(conf_matrix)


print("\nClassification Report:")
print(classification_report(y_test, y_pred))

90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Confusion Matrix:
[[ 243   94  120]
 [ 103  260  232]
 [  72  213 1542]]

Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.53      0.56       457
           1       0.46      0.44      0.45       595
           2       0.81      0.84      0.83      1827

    accuracy                           0.71      2879
   macro avg       0.62      0.60      0.61      2879
weighted avg       0.70      0.71      0.71      2879

